In [1]:
import os
os.environ['HF_HOME'] = "/Users/artemmerinov/data/backbones/huggingface" # before 

import timm
from timm.models import vision_transformer
from timesformer.models.vit import TimeSformer

import torch
import torch.nn as nn
import torchvision.models as models
from torch.profiler import profile, record_function, ProfilerActivity

/Users/artemmerinov/PycharmProjects/holoassist/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
vit_model = timm.create_model(
    model_name='timm/vit_base_patch16_224.augreg_in21k_ft_in1k',
    pretrained=True,
    num_classes=2000, 
)
vit_state = vit_model.state_dict()

for key in list(vit_state.keys()):
    vit_state[f"model.{key}"] = vit_state.pop(key)
    
timesformer_model = TimeSformer(
    img_size=224, 
    num_classes=2000, 
    num_frames=8, 
    attention_type='divided_space_time', 
    # pretrained_model="/Users/artemmerinov/.cache/torch/hub/checkpoints/jx_vit_base_p16_224-80ecf9dd.pth"
)
timesformer_model.load_state_dict(vit_state, strict=False)
timesformer_model_state = timesformer_model.state_dict()

In [14]:
inputs = torch.randn(5, 3, 8, 224, 224)

with profile(activities=[ProfilerActivity.CPU], profile_memory=True, record_shapes=True) as prof:
    timesformer_model(inputs)

STAGE:2024-04-30 19:41:19 55857:1682191 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2024-04-30 19:41:32 55857:1682191 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2024-04-30 19:41:32 55857:1682191 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


In [18]:
print(prof.key_averages().table(sort_by="cpu_memory_usage", row_limit=10))

----------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                        Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg       CPU Mem  Self CPU Mem    # of Calls  
----------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                 aten::empty         0.08%       8.420ms         0.08%       8.420ms      28.836us       3.85 Gb       3.85 Gb           292  
                aten::linear         0.02%       2.076ms        56.24%        6.000s      70.585ms       3.77 Gb           0 b            85  
                 aten::addmm        49.10%        5.238s        56.19%        5.994s      70.522ms       3.77 Gb       3.77 Gb            85  
                aten::matmul         0.02%       2.660ms         8.30%     885.000ms      18.438ms       3.03 Gb           0 b            48  